## Astra Schema

### Initialise Connection

In [ ]:
from astra_db import AstraSession

session = AstraSession().session

## Schema

### Create Tables

In [ ]:
#
# Communications Table
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS communications (
    communication_id TEXT,
    comm_date TIMESTAMP,
    customer_id TEXT,
    channel TEXT,
    category_group TEXT,
    category TEXT,
    activity_name TEXT,
    PRIMARY KEY ((communication_id), comm_date) );
    """
)

### Create CAP Tables

#### Daily Tables

**Columns**

* activity_name: assumed to be around 10 characters e.g. "mortgage" or "creditcard"
* comm_date: the day (bucket) date in the format YYYYMMDD; there is a new partition for each day.
* comm_time: the time a message is written in the format HHMMSSssss; there is a new row for each write message time.
* comm_offset_time: the time offset in the same format as comm_time; use this in conjunction with comm_offset_count to reduce the number of rows read.
* comm_offset_count: the number of messages from the start of the day to comm_offset_time.

There is no need to store the message id as we just want to count the number of messages in a given time period. To save on space, we store the time the message was written in a custom format that is smaller than a timestamp.


#### Weekly Tables
This table will store the number of messages sent on every day in a given year for a given activity.

**Columns**
* activity_name: assumed to be around 10 characters e.g. "mortgage" or "creditcard"
* comm_date: the day date in the format YYYYMMDD; there is a new row for each day in the week.
* comm_count: the number of messages sent for the given comm_date day.

Rather than store each message sent in a day for an activity, we store the total number of messages sent in a day. 
This significantly reduces the number of rows stored and the amount of work the application needs to do to get 
the total number of messages sent for a certain time period. 
The number of messages sent in a day can be obtained from the comm_by_activity_day table and used to populate this table.

In [ ]:
#
# Communications by Activity
#
# This table will store the number of messages sent in a day for a given activity.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS comm_by_activity_day (
        activity_name text,
        comm_date int, 
        comm_time int, 
        comm_offset_time int static,
        comm_offset_count int static,
        PRIMARY KEY ((activity_name, comm_date), comm_time)
    ) WITH CLUSTERING ORDER BY (comm_time DESC) AND default_time_to_live = 604800;
    """
)

#
# This table will store the number of messages sent on every day in a given year for a given activity.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS comm_by_activity_week (
        activity_name text,
        comm_date int,
        comm_count int,
        PRIMARY KEY ((activity_name), comm_date)
    ) WITH CLUSTERING ORDER BY (comm_date DESC) AND default_time_to_live = 7776000;
    """
)

## Drop Schema

In [ ]:
session.execute("DROP TABLE IF EXISTS communications")
session.execute("DROP TABLE IF EXISTS comm_by_activity_day")
session.execute("DROP TABLE IF EXISTS comm_by_activity_week")